In [ ]:
import requests
from bs4 import BeautifulSoup
import pyautogui

headers = {
    "User-Agent": "your_user_agent",
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3",
}

keyword = pyautogui.prompt("검색어를 입력하세요>>>")

rank = 1
done = False
# 헤더에 User-Agent를 추가하지 않으면 오류가 남(멈춰버림)


for page in range(1, 5):
    if done == True:
        break
    print(page, "번째 페이지입니다.")
    main_url = f"https://www.coupang.com/np/search?&q={keyword}&page={page}"
    response = requests.get(main_url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("a.search-product-link")  # select의 결과는 리스트 자료형
    print(links)
    for link in links:
        # 광고상품 제거
        if len(link.select("span.ad-badge-text")) > 0:
            print("광고상품입니다.")
        else:
            sub_url = "https://www.coupang.com/" + link.attrs["href"]
            response = requests.get(sub_url, headers=headers)
            html = response.text
            soup = BeautifulSoup(html, "html.parser")

            # 브랜드명은 있을 수도 있고, 없을 수도 있음
            # 중고상품의 경우 태그가 달라짐
            # try-except로 예외 처리

            try:
                brand_name = soup.select_one("a.prod-brand-name").text
            except:
                brand_name = ""

            brand_name = soup.select_one("a.prod-brand-name").text

            # 상품명
            product_name = soup.select_one("h2.prod-buy-header__title").text

            # 가격
            try:
                product_price = soup.select_one("span.total-price > strong").text
            except:
                product_price = 0
            print(brand_name, product_name, product_price)
            rank = rank + 1
            if rank > 100:
                done = True
                break